# Import Dependencies

In [6]:
from datetime import datetime
from io import BytesIO
from sqlalchemy import create_engine, inspect, text
from time import time

import json
import locale
import numpy as np
import pandas as pd
import requests

import requests
import time as t
from geoalchemy2 import Geometry, WKTElement

import warnings
warnings.filterwarnings("ignore")

# 1. Database setup

Notes: put your postgresql configuration in the config.json file

In [10]:
with open('config.json') as config:
    db_config = json.load(config)

db_user = db_config['DB_USER']
db_password = db_config['DB_PASSWORD']
db_host = db_config['DB_HOST']
db_port = db_config['DB_PORT']
db_name = db_config['DB_NAME']

db_engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
gh_inspector = inspect(db_engine)

# 2. Dataset 1: Greenhouse and energy information by designated generation facility

In [3]:
gh_energy_data_facts = [
    {
        "time_period_start": datetime(2023, 1, 7),
        "time_period_stop": datetime(2024, 6, 30),
        "dataset_id": "ID0243"
    },
    {
        "time_period_start": datetime(2022, 1, 7),
        "time_period_stop": datetime(2023, 6, 30),
        "dataset_id": "ID0083"
    },
    {
        "time_period_start": datetime(2021, 1, 7),
        "time_period_stop": datetime(2022, 6, 30),
        "dataset_id": "ID0082"
    },
    {
        "time_period_start": datetime(2020, 1, 7),
        "time_period_stop": datetime(2021, 6, 30),
        "dataset_id": "ID0081"
    },
    {
        "time_period_start": datetime(2019, 1, 7),
        "time_period_stop": datetime(2020, 6, 30),
        "dataset_id": "ID0080"
    },
    {
        "time_period_start": datetime(2018, 1, 7),
        "time_period_stop": datetime(2019, 6, 30),
        "dataset_id": "ID0079"
    },
    {
        "time_period_start": datetime(2017, 1, 7),
        "time_period_stop": datetime(2018, 6, 30),
        "dataset_id": "ID0078"
    },
    {
        "time_period_start": datetime(2016, 1, 7),
        "time_period_stop": datetime(2017, 6, 30),
        "dataset_id": "ID0077"
    },
    {
        "time_period_start": datetime(2015, 1, 7),
        "time_period_stop": datetime(2016, 6, 30),
        "dataset_id": "ID0076"
    },
    {
        "time_period_start": datetime(2014, 1, 7),
        "time_period_stop": datetime(2015, 6, 30),
        "dataset_id": "ID0075"
    }
]

gh_energy_table_name = "greenhouse_and_energy"

gh_energy_column_aliases = [
    {"reporting_entity": ["reportingentity", "reportingEntity", "controllingcorporation"]},
    {"facility_name" : ["facilityname", "facilityName"]},
    {"type": ["type"]},
    {"state": ["state"]},
    {"electricity_production_gj": ["electricityproductionGJ", "electricityProductionGJ"]},
    {"electricity_production_mwh" :["electricityproductionMWh", "electricityProductionMWh", "electricityProductionMwh"]},
    {"total_scope_1_emissions_t_co2_e": ["totalscope1emissionstCO2e", "totalScope1EmissionstCO2e", "scope1tCO2e"]},
    {"total_scope_2_emissions_t_co2_e": ["totalscope2emissionstCO2e", "totalScope2EmissionstCO2e", "totalScope2EmissionstCO2e2", "scope2tCO2e"]},
    {"total_emissions_t_co2_e": ["totalemissionstCO2e", "totalEmissionstCO2e"]},
    {"emission_intensity_t_co2_emwh": ["emissionintensitytCO2eMWh", "emissionIntensitytCO2eMWh", "emissionIntensitytMwh"]},
    {"grid_connected": ["gridconnected", "gridConnected", "gridConnected2"]},
    {"grid": ["grid"]},
    {"primary_fuel": ["primaryfuel", "primaryFuel"]},
    {"important_notes": ["importantnotes", "importantNotes"]},
    {"time_period_start": ["time_period_start"]},
    {"time_period_stop": ["time_period_stop"]},
    {"dataset_id": ["dataset_id"]} 
]

In [ ]:
def standardize_column_name(df, col_name_aliases):
    col_names = df.columns.tolist()
    col_map = {}

    for col in col_names:
        for alias_dict in col_name_aliases:
            alias = list(alias_dict.values())[0]
            if col in alias:
                col_map[col] = list(alias_dict.keys())[0]
                break

    return df.rename(columns=col_map)

total_row_inserted = 0
for index, fact in enumerate(gh_energy_data_facts):
    # read csv
    dataset_id = fact['dataset_id']
    url = f'https://api.cer.gov.au/datahub-public/v1/api/ODataDataset/NGER/dataset/{dataset_id}?select%3D%2A'
    response = requests.get(url)
    df = pd.DataFrame(response.json())

    # standardize column name
    df = standardize_column_name(df, gh_energy_column_aliases)

    # add columns: time_period, dataset_id
    df['time_period_start'] = fact['time_period_start']
    df['time_period_stop'] = fact['time_period_stop']
    df['dataset_id'] = fact['dataset_id']

    if index == 0:
        # Generate the CREATE TABLE statement
        create_table_statement = pd.io.sql.get_schema(df, gh_energy_table_name)
        
        # Print the generated statement
        print(create_table_statement)

        # Create table
        df.head(n=0).to_sql(name=gh_energy_table_name, con=db_engine, if_exists='replace')
    
    print(f'inserting table {fact['dataset_id']}...')
    t_start = time()
    df.to_sql(name=gh_energy_table_name, con=db_engine, if_exists='append')
    t_end = time()
    print(f'inserted table {fact['dataset_id']} in {t_end-t_start:10.3f} seconds')
    total_row_inserted = total_row_inserted + len(df)
    print(f'{total_row_inserted} rows inserted')
        

# 3. Dataset 2: Large-scale renewable energy data

## 3.1. 2001–2024 Accredited power stations data

In [ ]:
url = "https://cer.gov.au/document/historical-accredited-power-stations-and-projects-0"
df = pd.read_csv(url, encoding='ISO-8859-1')
df.head()

In [ ]:
# Rename column names
df = df.rename(columns={
    'Accreditation code': 'accreditation_code',
    'Power station name': 'power_station_name',
    'State': 'state',
    'Installed capacity': 'installed_capacity',
    'Postcode': 'postcode',
    'Fuel source(s)': 'fuel_sources',
    'Accreditation start date': 'accreditation_start_date',
    'Suspension status': 'suspension_status',
    'Baseline (MWh)': 'baseline_mwh',
    'Comment': 'comment' 
})
df.columns

In [ ]:
# check data types
print(df.dtypes)

In [ ]:
# convert data types

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
df['baseline_mwh'] = df['baseline_mwh'].apply(lambda x: locale.atoi(str(x)) if isinstance(x, str) else x)

df['accreditation_start_date'] = pd.to_datetime(df['accreditation_start_date'], format='%d/%m/%Y')

In [ ]:
# save to database
table_name='accredited_power_stations_data'
df.to_sql(name=table_name, con=db_engine, if_exists='replace')

## 3.2. 2024 total LGCs and capacity of accredited power stations

In [ ]:
url = "https://cer.gov.au/document/total-lgcs-and-capacity-accredited-power-stations-2024"
df = pd.read_csv(url, encoding='ISO-8859-1')
df.head()

In [ ]:
# Rename column names
df = df.rename(columns={
    'Total LGCs in the REC Registry': 'total_lgcs_in_the_rec_registry',
    'MW of approved power stations (since 1 Jan 2024)': 'mw_of_approved_power_stations_since_1_jan_2024',
    'Approved power stations (since 1 Jan 2024)': 'approved_power_stations_since_1_jan_2024',
    'As at': 'as_at'
})
df.columns

In [ ]:
# check data types
print(df.dtypes)

In [ ]:
# convert data types

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
df['mw_of_approved_power_stations_since_1_jan_2024'] = df['mw_of_approved_power_stations_since_1_jan_2024'].apply(lambda x: locale.atof(str(x)) if isinstance(x, str) else x)

df['as_at'] = pd.to_datetime(df['as_at'], format='%d/%m/%Y')

In [ ]:
# save to database
table_name='total_lgcs_and_capacity_of_accredited_power_stations'
df.to_sql(name=table_name, con=db_engine, if_exists='replace')

## 3.3. Committed power stations

In [12]:
url = "https://cer.gov.au/document/power-stations-and-projects-committed"
df = pd.read_csv(url, encoding='ISO-8859-1')
df.head()

,Project Name,State,MW Capacity,Fuel Source,Committed Date (Month/Year)
0,East Rockingham Resource Recovery Facility,WA,29.0,Biomass,Dec-2019
1,Mangalore Renewable Energy Project,VIC,5.0,Solar,Sep-2021
2,Orange Community Renewable Energy Park,NSW,5.0,Solar,Jul-2022
3,Moorebank Logistics Park,NSW,60.0,Solar,Sep-2022
4,Wangaratta Solar Farm,VIC,40.0,Solar,Jul-2023


In [16]:
# Rename column names
df = df.rename(columns={
    'Project Name': 'project_name',
    'State ': 'state',
    'MW Capacity': 'mw_capacity',
    'Fuel Source': 'fuel_source',
    'Committed Date (Month/Year)': 'committed_date'
})
df.columns

Index(['project_name', 'state', 'mw_capacity', 'fuel_source',
       'committed_date'],
      dtype='object')

In [22]:
# check data types
print(df.dtypes)

project_name              object
state                     object
mw_capacity              float64
fuel_source               object
committed_date    datetime64[ns]
dtype: object


In [20]:
# convert data types
df["committed_date"] = pd.to_datetime(df["committed_date"], format="%b-%Y")

In [24]:
# save to database
table_name='committed_power_stations'
df.to_sql(name=table_name, con=db_engine, if_exists='replace')

35

In [ ]:
## 3.4. Probable power stations

In [26]:
url = "https://cer.gov.au/document/power-stations-and-projects-probable"
df = pd.read_csv(url, encoding='ISO-8859-1')
df.head()

,Project Name,State,MW Capacity,Fuel Source
0,Barnawartha Solar Farm,VIC,64.0,Solar
1,Barwon solar farm,VIC,250.0,Solar
2,Boddington Giga Energy,WA,400.0,Solar
3,Bulli Creek Solar project Stage 1,QLD,775.0,Solar
4,Bullyard Solar Farm,QLD,100.0,Solar


In [30]:
# Rename column names
df = df.rename(columns={
    'Project Name': 'project_name',
    'State ': 'state',
    'MW Capacity': 'mw_capacity',
    'Fuel Source': 'fuel_source',
})
df.columns

Index(['project_name', 'state', 'mw_capacity', 'fuel_source'], dtype='object')

In [32]:
# check data types
print(df.dtypes)

project_name     object
state            object
mw_capacity     float64
fuel_source      object
dtype: object


In [34]:
# save to database
table_name='probable_power_stations'
df.to_sql(name=table_name, con=db_engine, if_exists='replace')

49

# 4. Dataset 3:  Australian Bureau of Statistic Data

In [ ]:
# base url for the data file

url = "https://www.abs.gov.au/methodologies/data-region-methodology/2011-24/14100DO0001_2011-24.xlsx"

In [ ]:
# function to clean column names

def clean_column_names(df):
    # Clean the column names
    df.columns = df.columns.str.lower()  # Convert to lowercase
    df.columns = df.columns.str.replace(' ', '_')  # Replace spaces with underscores
    df.columns = df.columns.str.replace('-', '_')  # Replace dash with underscores
    df.columns = df.columns.str.replace('no.', 'integer')  # Replace dash with underscores
    df.columns = df.columns.str.replace('%', 'pct')  # Replace % with 'pct'
    df.columns = df.columns.str.replace('[^a-z0-9_]', '', regex=True)  # Remove special characters
    return df

In [ ]:
# function to convert data types

def convert_abs_datatype(df):
    locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
    
    unchanged_col = ["code", "label", "year"]
    
    # Opt-in to the future behavior of replace
    pd.set_option('future.no_silent_downcasting', True)
    
    for col in list(df.columns):
        if col not in unchanged_col:
            if "integer" in col:
                df[col] = df[col].replace('-', np.nan)
                df[col] = df[col].apply(lambda x: locale.atoi(str(x)) if isinstance(x, str) else x)
                df[col] = df[col].astype("Int64")
            else:
                df[col] = df[col].replace('-', np.nan)
                df[col] = df[col].apply(lambda x: locale.atof(str(x)) if isinstance(x, str) else x)
    return df

## 4.1. Estimated resident population - year ended 30 June

In [ ]:
table_name = 'estimated_resident_population'

### 4.1.1. Table 1 (For exploratory)

In [ ]:
df = pd.read_excel(url, sheet_name="Table 1", skiprows=6, usecols="A:C,D:L")

# check table
df.head()

In [ ]:
# clean column names
df = clean_column_names(df)    
df.columns

In [ ]:
# convert data type
df = convert_abs_datatype(df)
print(df.dtypes)

In [ ]:
# adding geo_scope identifier
df['geo_scope'] = 1
df['geo_scope'] = df['geo_scope'].astype(int)

In [ ]:
# re-check table
df.head()

In [ ]:
# store data to database
df.to_sql(name=table_name, con=db_engine, if_exists='replace')

In [ ]:
# the next stage will go through the same steps, to not repeating ourselves, we can use this function:

def dump_data(sheet, skip_rows, use_cols, geo_scope, postgres_table_name, store_mode):
    # read file
    df = pd.read_excel(url, sheet_name=sheet, skiprows=skip_rows, usecols=use_cols)

    # clean column names
    df = clean_column_names(df)  

    # convert data type
    df = convert_abs_datatype(df)

    # adding geo_scope identifier
    df['geo_scope'] = geo_scope
    df['geo_scope'] = df['geo_scope'].astype(int)

    # store data to database
    df.to_sql(name=postgres_table_name, con=db_engine, if_exists=store_mode)

### 4.1.2. Table 2 and table 3

In [ ]:
col_span="A:C,D:L"
dump_data("Table 2", 6, col_span, 2, table_name, "append")
dump_data("Table 3", 6, col_span, 3, table_name, "append")

## 4.2. Estimated resident population - Males - year ended 30 June

In [ ]:
table_name = 'estimated_resident_population_males'
col_span="A:C,M:AV"

# Table 1
dump_data("Table 1", 6, col_span, 1, table_name, "replace")
# Table 2
dump_data("Table 2", 6, col_span, 2, table_name, "append")
# Table 3
dump_data("Table 3", 6, col_span, 3, table_name, "append")

## 4.3 Estimated resident population - Females - year ended 30 June

In [ ]:
table_name = 'estimated_resident_population_females'
col_span="A:C,AW:CF"

# Table 1
dump_data("Table 1", 6, col_span, 1, table_name, "replace")
# Table 2
dump_data("Table 2", 6, col_span, 2, table_name, "append")
# Table 3
dump_data("Table 3", 6, col_span, 3, table_name, "append")

## 4.4 Estimated resident population - Persons - year ended 30 June

In [ ]:
table_name = 'estimated_resident_population_persons'
col_span="A:C,CG:DP"

# Table 1
dump_data("Table 1", 6, col_span, 1, table_name, "replace")
# Table 2
dump_data("Table 2", 6, col_span, 2, table_name, "append")
# Table 3
dump_data("Table 3", 6, col_span, 3, table_name, "append")

## 4.5. Births and deaths - year ended 31 December

In [ ]:
table_name = 'births_and_deaths'
col_span="A:C,DQ:DT"

# Table 1
dump_data("Table 1", 6, col_span, 1, table_name, "replace")
# Table 2
dump_data("Table 2", 6, col_span, 2, table_name, "append")
# Table 3
dump_data("Table 3", 6, col_span, 3, table_name, "append")

## 4.6. Internal and overseas migration - year ended 30 June

In [ ]:
table_name = 'internal_and_overseas_migration'
col_span="A:C,DU:DZ"

# Table 1
dump_data("Table 1", 6, col_span, 1, table_name, "replace")
# Table 2
dump_data("Table 2", 6, col_span, 2, table_name, "append")

## 4.7. Aboriginal and Torres Strait Islander Peoples - Census

In [ ]:
table_name = 'aboriginal_and_torres_strait_islander_peoples'
col_span="A:C,EA:EB"

# Table 1
dump_data("Table 1", 6, col_span, 1, table_name, "replace")
# Table 2
dump_data("Table 2", 6, col_span, 2, table_name, "append")

## 4.8. Overseas born population

In [ ]:
table_name = 'overseas_born_population'
col_span="A:C,EC:EM"

# Table 1
dump_data("Table 1", 6, col_span, 1, table_name, "replace")
# Table 2
dump_data("Table 2", 6, col_span, 2, table_name, "append")

## 4.9. Religious affiliation

In [ ]:
table_name = 'religious_affiliation'
col_span="A:C,EN:EU"

# Table 1
dump_data("Table 1", 6, col_span, 1, table_name, "replace")
# Table 2
dump_data("Table 2", 6, col_span, 2, table_name, "append")

## 4.10. Australian citizenship

In [ ]:
table_name = 'australian_citizenship'
col_span="A:C,EV:FA"

# Table 1
dump_data("Table 1", 6, col_span, 1, table_name, "replace")
# Table 2
dump_data("Table 2", 6, col_span, 2, table_name, "append")

## 4.11. Speaks a language other than English at home

In [ ]:
table_name = 'speaks_other_than_english_at_home'
col_span="A:C,FB:FC"

# Table 1
dump_data("Table 1", 6, col_span, 1, table_name, "replace")
# Table 2
dump_data("Table 2", 6, col_span, 2, table_name, "append")

## 4.12. Australian Defence Force service - Persons aged 15 years and over

In [ ]:
table_name = 'australian_defence_force_service'
col_span="A:C,FD:FG"

# Table 1
dump_data("Table 1", 6, col_span, 1, table_name, "replace")
# Table 2
dump_data("Table 2", 6, col_span, 2, table_name, "append")

# 5. Data Augmentation

In [36]:
def get_state_coordinates(df, state_col='state', wait=1):
    # Extract unique states
    state_list = df[state_col].dropna().unique().tolist()

    # Initialize dictionary to store results
    state_coords = {}

    # Nominatim API setup
    base_url = 'https://nominatim.openstreetmap.org/search'
    headers = {'User-Agent': 'COMP5339'}

    # Query OpenStreetMap for each state
    for state in state_list:
        params = {'q': state, 'format': 'json'}
        t.sleep(wait)  # avoid overloading the service
        try:
            response = requests.get(base_url, params=params, headers=headers)
            if response.status_code == 200:
                data = response.json()
                if data:
                    lat = float(data[0]['lat'])
                    lon = float(data[0]['lon'])
                    state_coords[state] = (lat, lon)
                else:
                    state_coords[state] = (None, None)
            else:
                state_coords[state] = (None, None)
        except:
            state_coords[state] = (None, None)

    return state_coords


In [42]:
# Function to return the latitude and longitude of the address using OpenStreetMap Nominatim API
def geocode_osm(name, state, state_coords=None, wait=1):
    # Base URL for OpenStreetMap Nominatim API
    base_url = 'https://nominatim.openstreetmap.org/search'
    headers = {'User-Agent': 'COMP5339'}

    # Step 1: Try geocoding the name only
    params = {'q': name, 'format': 'json'}
    t.sleep(wait)
    try:
        response = requests.get(base_url, params=params, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data:
                return float(data[0]['lat']), float(data[0]['lon'])
    except:
        pass  # silently ignore errors

    # Step 2: If name fails, check if state exists in the provided dictionary
    if state_coords and state in state_coords:
        return state_coords[state]

    # Step 3: If all fails, return None
    return None, None

In [44]:
# Query the table and load into DataFrame
table_name = 'accredited_power_stations_data'
query = f"SELECT * FROM {table_name}"

# Access the power stations table using pandas
power_station_df = pd.read_sql(query, con=db_engine)

# Display the first few rows of the table
power_station_df.head()

,index,accreditation_code,power_station_name,state,installed_capacity,postcode,fuel_sources,accreditation_start_date,suspension_status,baseline_mwh,comment
0,0,SRPXQLG5,Varsity Views - Solar w SGU - QLD,QLD,0.2109,4227,Solar,2024-12-18,Unsuspended,0,None
1,1,SRPYNS51,NNSWLHD-Byron Central Hospital - Solar - NSW,NSW,0.7203,2481,Solar,2024-12-12,Unsuspended,0,None
2,2,SRPXQLG2,Springwood Terrace Care Community - Solar w SG...,QLD,0.1890,4127,Solar,2024-12-10,Unsuspended,0,None
3,3,SRPYNS45,Bunnings Bennetts Green - Solar - NSW,NSW,0.2800,2290,Solar,2024-12-03,Unsuspended,0,None
4,4,SRPYNS46,Tuggerah Home MSB1 254kW - Solar - NSW,NSW,0.2540,2259,Solar,2024-12-02,Unsuspended,0,None


In [46]:
# Get unique state coordinates
state_coords = get_state_coordinates(power_station_df, state_col='state', wait=1)

state_coords

{'QLD': (-22.1646782, 144.5844903),
 'NSW': (-31.8759835, 147.2869493),
 'VIC': (41.9302021, 2.2545943),
 'SA': (25.6242618, 42.3528328),
 'WA': (-25.2303005, 121.0187246),
 'ACT': (-35.4883502, 149.0026942),
 'TAS': (-42.035067, 146.6366887),
 'NT': (-19.8516101, 133.2303375)}

In [ ]:
# Apply geocode_osm to each row and unpack results into 'Latitude' and 'Longitude'
power_station_df['Latitude'], power_station_df['Longitude'] = zip(*power_station_df.apply(lambda row: geocode_osm(row['power_station_name'], row['state'], state_coords=state_coords), axis=1))

power_station_df.head()

In [ ]:
# Test connection and check PostGIS status using text() for SQL
with db_engine.connect() as connection:
    result = connection.execute(text("SELECT PostGIS_full_version();"))
    postgis_version = result.fetchone()
    print(f"PostGIS Version: {postgis_version[0]}")

In [ ]:
# Convert lat/lon to PostGIS Point
power_station_df['geom'] = power_station_df.apply(
    lambda row: WKTElement(f"POINT({row['Longitude']} {row['Latitude']})", srid=4326),
    axis=1
)

power_station_df.head()

In [ ]:
table_name = 'spatial_power_stations_data'

# store data to database
power_station_df.to_sql(name=table_name, con=db_engine, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid=4326)})

In [ ]:
# Check Postgis
query = text("SELECT power_station_name, ST_AsText(geom) AS geom_wkt FROM spatial_power_stations_data LIMIT 5;")
df_check = pd.read_sql(query, con=db_engine)
print(df_check)

# 6. PostgreSQL